In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras import Model,layers


class CNN_WORK(Model):
    def __init__(self):
        super(CNN_WORK,self).__init__()
        self.conv1 = layers.Conv2D(32,kernel_size=5,activation=tf.nn.relu)
        self.maxpool1 = layers.MaxPool2D(2,strides=2)
        
        self.conv2 = layers.Conv2D(64,kernel_size=3,activation=tf.nn.relu)
        self.maxpool2 = layers.MaxPool2D(2,strides=2)
        
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(1024)
        self.dropout = layers.Dropout(rate=0.5)
        self.out = layers.Dense(2)
    
    def call(self,x,is_training=False):
       # x = tf.reshape(x,[-1,64,64,3])
        
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x,training=is_training)
        x = self.out(x)
        
        
        if not is_training:
            x = tf.nn.softmax(x)
        return x
model = CNN_WORK()

    

In [2]:


cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('C:\\Users\Wuhuipeng\AppData\Local\Programs\Python\Python36\Lib\site-packages\cv2\data/haarcascade_frontalface_alt.xml')

while True:
    ret,frame = cap.read()

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=5,minSize=(5,5))

    for (x,y,z,t) in faces:
        img = frame[y:y+t,x:x+z]
        try:
            img = cv2.resize(img,(64,64))
            img = tf.cast(img,tf.float32) / 255.0
            img = tf.reshape(img,[-1,64,64,3])
        
            pred = model(img)
            pred = tf.argmax(pred,axis=1).numpy()
        except:
            pass
        if(pred[0]==1):
            cv2.putText(frame,"wuhuipeng",(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.2,(255,255,0),2)
        
        cv2.rectangle(frame,(x,y),(x+z,y+t),(0,255,0),2)
    cv2.imshow('find faces',frame)
    if cv2.waitKey(1)&0xff ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()